# Imports

In [ ]:
import os
import anndata
import seaborn as sns
from sklearn import preprocessing
from Functions import run_phenograph, get_folder_names, ad_dimred_setup, test_UMAP_parameters, plot_umap, plot_random_organoids_cluster, run_slingshot, save_adata

# User input

In [ ]:
load_dir = "/Your/Absolute/Path"
save_dir = os.path.join(os.path.sep,*get_folder_names(load_dir)[:-1])
plot_save_dir = save_dir.replace("2_Results","3_Plots")

remove_features = ["DAPI"] # Remove all common features with input str in their name for dimension reduction

# Set up plotting parameters
fig_param = {"axes.spines.right": False,
                  "axes.spines.top": False,
                  "axes.spines.left": True,
                  "axes.spines.bottom": True,
                  "pdf.fonttype": 42,
                  "font.size": 6,
                  "axes.labelsize": 6,
                  "axes.titlesize": 8,
                  "xtick.labelsize": 6,
                  "ytick.labelsize": 6,
                  "legend.fontsize": 6}

sns.set_theme(rc = fig_param, style = "ticks")

# Color palette for time course experiments
palette_tc = {
     'Day 1':"#009392",
     'Day 2':"#26A789",
     'Day 3':"#5AB985",
     'Day 4':"#9CCB86",
     'Day 5':"#CFDA94",
     'Day 6':"#EAD290",
     'Day 7':"#EEB479", 
     'Day 8':"#EA9473",
     'Day 9':"#DF7575",
     'Day 10':"#CF597E"
               }

# Load & misc
Loads AnnData object and sets up an AnnData object for dimensionality reduction based on common features across all organoids not part of remove_features

In [ ]:
ad = anndata.read_h5ad(load_dir)

ad, ad_dimred = ad_dimred_setup(ad = ad,
                                remove_features = remove_features # List of keywords to be removed from features
                               )

# UMAP

### UMAP parameter testing
Test UMAP parameters for various combinations of n_neighbors and min_distances.

In [ ]:
ad = test_UMAP_parameters(ad = ad,
                          ad_dimred = ad_dimred,
                          feature_to_plot = "Other", # Feature to use as hue for UMAP visualization
                          plot_save_dir = plot_save_dir,
                          neighbors = [50, 100], # List of neighbor values to test. See UMAP package for further details
                          distances = [0.05, 0.10, 0.20, 0.30], # List of distance values to test. See UMAP package for further details
                          save_plot = True)

### Plot UMAP
Plot UMAP for selected features.

In [ ]:
ad = plot_umap(ad = ad,
          ad_dimred = ad_dimred,
          plot_feat = ["Other", "DAPI_mean", "area"], # List of features to use as hue for UMAP visualization
          n_neighbors = 50, # Integer of neighbor parameter. See UMAP package for further details
          min_dist = 0.05, # Float of min_dist parameter. See UMAP package for further details
          plot_save_dir = plot_save_dir,
          save_plot = True)

# Clustering

### Phenograph
Run PhenoGraph clustering on minmax normalized and log1p-transformed data.

In [ ]:
run_phenograph(ad = ad,
               ad_dimred = ad_dimred,
               k = 5, # Integer for number of nearest neighbors.
               plot_save_dir = plot_save_dir,
               save_plot = True)

### Plot n random organoids per cluster
Plot n_organoids random organoids for each PhenoGraph cluster.

In [ ]:
plot_random_organoids_cluster(ad = ad,
                              n_organoids = 10, # Number of randomly selected organoids to visualize. If greater than number of organoids within that cluster, organoids are resampled.
                              plot_save_dir = plot_save_dir,
                              seed = 0,
                              save_plot = False)

# Pseudotime
Run Slingshot analysis for trajectory inference.

In [ ]:
slingshot_object = run_slingshot(ad = ad,
                                 start_cluster = 5, # Cluster name as a starting node for trajectory inference.
                                 num_epochs = 3, # Number of iterations
                                 plot_save_dir = plot_save_dir,
                                 save_plot = False)

# Save AD

In [ ]:
# Save Slingshot results to AnnData object
ad.obs["Pseudotime"] = preprocessing.MinMaxScaler().fit_transform(slingshot_object.unified_pseudotime.reshape(-1, 1))
for lin in range(len(slingshot_object.lineages)):
    ad.obs["slingshot_weight_lineage_"+str(lin)]= [x[lin]for x in slingshot_object.cell_weights]

# Save AnnData object
save_adata(save_dir, "5_Pseudotime", ad)